# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
vacation_df = pd.read_csv('/Users/novakradovic/Git/python_api_challenge/output/city_data.csv')
vacation_df


,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,ous,ru,60.9058,61.5186,14.98,70.0,36.0,1.82,1.626900e+09
1,1,punta arenas,cl,-53.1500,-70.9167,5.06,81.0,90.0,6.69,1.626900e+09
2,2,mataura,pf,-46.1927,168.8643,6.93,97.0,100.0,1.47,1.626900e+09
3,3,longyearbyen,sj,78.2186,15.6401,5.91,65.0,20.0,6.69,1.626900e+09
4,4,ushuaia,ar,-54.8000,-68.3000,0.81,80.0,75.0,2.06,1.626900e+09
...,...,...,...,...,...,...,...,...,...,...
545,545,cobija,bo,-11.0267,-68.7692,29.10,48.0,0.0,3.09,1.626900e+09
546,546,northam,au,51.0333,-4.2167,22.89,76.0,36.0,1.25,1.626900e+09
547,547,elizabeth city,us,36.2946,-76.2511,34.03,62.0,20.0,2.24,1.626900e+09
548,548,cao bang,vn,22.6667,106.2500,24.04,96.0,100.0,0.99,1.626900e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
fig = gmaps.figure()
locations = vacation_df[['Latitude', 'Longitude']]
weights = vacation_df['Humidity']
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
new_df = vacation_df.loc[(vacation_df['Max Temperature']<27)&(vacation_df['Max Temperature']>22)
                         &(vacation_df['Cloudiness']==0)&(vacation_df['Wind Speed']<16)
                        &(vacation_df['Humidity']<60)].reset_index(drop=True)
hotel_df = new_df.drop(columns=['Unnamed: 0'])
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date
0,manono,cd,-7.3000,27.4167,25.15,31.0,0.0,2.23,1.626900e+09
1,tarija,bo,-21.5355,-64.7296,24.95,13.0,0.0,2.06,1.626900e+09
2,itumbiara,br,-18.4192,-49.2153,25.25,27.0,0.0,1.78,1.626900e+09
3,makrinitsa,gr,39.4000,22.9833,25.72,51.0,0.0,4.65,1.626900e+09
4,formosa,ar,-26.1775,-58.1781,22.99,40.0,0.0,2.57,1.626900e+09
5,shahrud,ir,36.4182,54.9763,24.91,22.0,0.0,2.36,1.626900e+09
6,shelburne,ca,44.0787,-80.2041,23.09,33.0,0.0,4.96,1.626900e+09
7,ahtopol,bg,42.1000,27.9500,25.26,52.0,0.0,5.35,1.626900e+09
8,qostanay,kz,53.1667,63.5833,22.16,56.0,0.0,4.00,1.626900e+09
9,kigoma,tz,-4.8769,29.6267,23.44,55.0,0.0,1.71,1.626900e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
import pprint
hotel_df['Hotel Name'] = ''
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    long = row['Longitude']
    params["location"] = f"{lat},{long}"
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['results'][0]["name"]
        
    except:
        print("Hotel within 5000m not found... skipping.")

# print(json.dumps(name_address, indent=4, sort_keys=True))
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,manono,cd,-7.3000,27.4167,25.15,31.0,0.0,2.23,1.626900e+09,Hotel hewa bora
1,tarija,bo,-21.5355,-64.7296,24.95,13.0,0.0,2.06,1.626900e+09,Hotel Los Parrales
2,itumbiara,br,-18.4192,-49.2153,25.25,27.0,0.0,1.78,1.626900e+09,Hotel Beira Rio
3,makrinitsa,gr,39.4000,22.9833,25.72,51.0,0.0,4.65,1.626900e+09,Valeni Boutique Hotel & Spa
4,formosa,ar,-26.1775,-58.1781,22.99,40.0,0.0,2.57,1.626900e+09,Howard Johnson by Wyndham Formosa Casino
5,shahrud,ir,36.4182,54.9763,24.91,22.0,0.0,2.36,1.626900e+09,Saderat Bank Hotel
6,shelburne,ca,44.0787,-80.2041,23.09,33.0,0.0,4.96,1.626900e+09,Shelburne Motel
7,ahtopol,bg,42.1000,27.9500,25.26,52.0,0.0,5.35,1.626900e+09,Eskada Beach
8,qostanay,kz,53.1667,63.5833,22.16,56.0,0.0,4.00,1.626900e+09,Hotel Fessla
9,kigoma,tz,-4.8769,29.6267,23.44,55.0,0.0,1.71,1.626900e+09,Sunset Vista Hotel


In [52]:
import datetime
from datetime import date
for i in hotel_df['Date']:
    timestamp = date.fromtimestamp(i)

hotel_df = hotel_df.drop(columns=['Date'])
hotel_df['Clean Date'] = timestamp
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Clean Date
0,manono,cd,-7.3000,27.4167,25.15,31.0,0.0,2.23,2021-07-21
1,tarija,bo,-21.5355,-64.7296,24.95,13.0,0.0,2.06,2021-07-21
2,itumbiara,br,-18.4192,-49.2153,25.25,27.0,0.0,1.78,2021-07-21
3,makrinitsa,gr,39.4000,22.9833,25.72,51.0,0.0,4.65,2021-07-21
4,formosa,ar,-26.1775,-58.1781,22.99,40.0,0.0,2.57,2021-07-21
5,shahrud,ir,36.4182,54.9763,24.91,22.0,0.0,2.36,2021-07-21
6,shelburne,ca,44.0787,-80.2041,23.09,33.0,0.0,4.96,2021-07-21
7,ahtopol,bg,42.1000,27.9500,25.26,52.0,0.0,5.35,2021-07-21
8,qostanay,kz,53.1667,63.5833,22.16,56.0,0.0,4.00,2021-07-21
9,kigoma,tz,-4.8769,29.6267,23.44,55.0,0.0,1.71,2021-07-21


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig_2 = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations)
fig_2.add_layer(markers)
fig_2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [45]:
# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))